In [31]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [32]:
path_to_data = "/scratch/ojomoleye/dukedata/"
training_groups =  path_to_data + "trainingset_clingroups.csv"
training_files = path_to_data + "trainingset_paths_studies_views.csv"
traingroups = pd.read_csv(training_groups)
trainfiles = pd.read_csv(training_files)

# Data description
 - No of patients
 - No of studies 
 - Normal, Benign, Cancerous and Actionable studies (patients)

In [41]:
num_train_pts = traingroups.PatientID.nunique()
num_train_pics = len(traingroups)
num_train_studies = traingroups.StudyUID.nunique()

In [43]:
print('there are {} patients, {} studies and {} total views in the training group'.format(
    num_train_pts, num_train_studies, num_train_pics))

there are 4362 patients, 4838 studies and 19148 total views in the training group


In [44]:
traingroups.head()

,PatientID,StudyUID,View,Normal,Actionable,Benign,Cancer
0,DBT-P00013,DBT-S00163,rmlo,0,0,1,0
1,DBT-P00023,DBT-S04378,lcc,0,1,0,0
2,DBT-P00023,DBT-S04378,lmlo,0,1,0,0
3,DBT-P00023,DBT-S04378,rcc,0,1,0,0
4,DBT-P00023,DBT-S04378,rmlo,0,1,0,0


The file paths in the csv that comes with the download isn't exactly the same as with our download. The file name is different but the directories from PatientID to the penultimate correspond.

In [45]:
trainfiles.head()
trainfiles['real_path'] = trainfiles.classic_path.str.split("/").str[1:4].apply(lambda x: path_to_data + "/".join(x) + "/*.dcm")

In [46]:
trainfiles.head()

,PatientID,StudyUID,View,descriptive_path,classic_path,real_path
0,DBT-P00013,DBT-S00163,rmlo,Breast-Cancer-Screening-DBT/DBT-P00013/01-01-2000-DBT-S00163-MAMMO DIAGNOSTIC DIGITAL BILATERAL-56865/20566.000000-32081/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00013/1.2.826.0.1.3680043.8.498.97194784071397240980122342729639556865/1.2.826.0.1.3680043.8.498.97979602815077649368346148322369132081/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00013/1.2.826.0.1.3680043.8.498.97194784071397240980122342729639556865/1.2.826.0.1.3680043.8.498.97979602815077649368346148322369132081/*.dcm
1,DBT-P00023,DBT-S04378,lcc,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19711.000000-14072/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.12136582480949936067549454937434114072/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.12136582480949936067549454937434114072/*.dcm
2,DBT-P00023,DBT-S04378,lmlo,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19710.000000-51654/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.90045035130681803298603121403642951654/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.90045035130681803298603121403642951654/*.dcm
3,DBT-P00023,DBT-S04378,rcc,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19712.000000-09387/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.10822555886306795549982251172752709387/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.10822555886306795549982251172752709387/*.dcm
4,DBT-P00023,DBT-S04378,rmlo,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19713.000000-61579/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.29938515490857039234478950570192161579/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.29938515490857039234478950570192161579/*.dcm


In [47]:
len(trainfiles) == trainfiles.real_path.nunique() 
#There are no duplicates in the real paths. Should be able to use os to locate the file with the wildcard at file name since only one file per sub sub folder

True

In [50]:
cancer_exams = traingroups[traingroups.Cancer == 1].StudyUID.nunique()
benign_exams = traingroups[traingroups.Benign == 1].StudyUID.nunique()
actionable_exams = traingroups[traingroups.Actionable == 1].StudyUID.nunique()
normal_exams = traingroups[traingroups.Normal == 1].StudyUID.nunique()

In [52]:
print(
    "{} unique patients, \n{} unique exams, \n{} cancer exams, \n{} benign exams, \n{} actionable exams, \n{} normal exams".format(
        num_train_pts, num_train_studies, cancer_exams, benign_exams, actionable_exams, normal_exams
    )
)

4362 unique patients, 
4838 unique exams, 
39 cancer exams, 
62 benign exams, 
179 actionable exams, 
4558 normal exams


Actionable - studies that resulted in further imaging examinaton based on a mass or acrchitectural distortion noted in the study report. "Specifically, we included studies that had a recommendation for a further imaging examination based on a mass or architectural distortion noted in the study report; did not result in a biopsy; had LCC, LMLO, RCC, and RMLO reconstruction views available; and did not contain spot compression among the 4 views."


Benign - containing benign masses or arthitectural distortions biopsied based on this DBT exam."Specifically, in this group we included studies that had a BI-RADS score of 0, 4, 4A, 4B, 4C, or 5; had a mass or architectural distortion that was seen in the DBT imaging study in question that was identified using laterality and/or location noted in a related breast pathology report and was biopsied; had benign results of all biopsies per the pathology reports; and a radiologist was able to retrospectively locate at least 1 of the biopsied benign masses or architectural distortions in the reconstruction views from the study. One study for which the biopsied mass was visible only on spot compression views was excluded."

In [53]:
trainfiles.head()

,PatientID,StudyUID,View,descriptive_path,classic_path,real_path
0,DBT-P00013,DBT-S00163,rmlo,Breast-Cancer-Screening-DBT/DBT-P00013/01-01-2000-DBT-S00163-MAMMO DIAGNOSTIC DIGITAL BILATERAL-56865/20566.000000-32081/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00013/1.2.826.0.1.3680043.8.498.97194784071397240980122342729639556865/1.2.826.0.1.3680043.8.498.97979602815077649368346148322369132081/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00013/1.2.826.0.1.3680043.8.498.97194784071397240980122342729639556865/1.2.826.0.1.3680043.8.498.97979602815077649368346148322369132081/*.dcm
1,DBT-P00023,DBT-S04378,lcc,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19711.000000-14072/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.12136582480949936067549454937434114072/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.12136582480949936067549454937434114072/*.dcm
2,DBT-P00023,DBT-S04378,lmlo,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19710.000000-51654/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.90045035130681803298603121403642951654/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.90045035130681803298603121403642951654/*.dcm
3,DBT-P00023,DBT-S04378,rcc,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19712.000000-09387/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.10822555886306795549982251172752709387/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.10822555886306795549982251172752709387/*.dcm
4,DBT-P00023,DBT-S04378,rmlo,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19713.000000-61579/1-1.dcm,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.29938515490857039234478950570192161579/1-1.dcm,/scratch/ojomoleye/dukedata/DBT-P00023/1.2.826.0.1.3680043.8.498.46740855673424283641249631860253820650/1.2.826.0.1.3680043.8.498.29938515490857039234478950570192161579/*.dcm
